In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
from IPython.core.debugger import set_trace
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from torch.utils.data import Dataset, DataLoader
from skimage import io,transform
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
import torchvision
import matplotlib.pyplot as plt


In [0]:
'''import os
df = pd.read_csv('/content/drive/My Drive/Resonance/ChiefData.csv')
l = []
root_dir = '/content/drive/My Drive/Resonance/Images'
print(len(df))
new_df = df.copy()
for i in range(len(list(df.iloc[:,0]))):
  spec_name = 'Num.'+str(df.iloc[i,0])+'.jpg'
  img_name = os.path.join(root_dir,spec_name)
  try: 
    io.imread(img_name)
  except:
    new_df = new_df.drop(i)
print(len(new_df))
new_df.to_csv('/content/drive/My Drive/Resonance/ChiefData.csv',index=False)'''

1525
1510


In [0]:
import os
from PIL import Image


class ImageDataset(Dataset):
  def __init__(self,df,root_dir,transform=None):
    self.df = df
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.df)
  
  def __getitem__(self,idx):
    spec_name = 'Num.'+str(self.df.iloc[idx,0])+'.jpg'
    img_name = os.path.join(self.root_dir,spec_name)
    image = Image.open(img_name).convert('RGB')
    ydata = self.df.iloc[idx,1]
    ydata = np.array(ydata)
    ydata = ydata.astype(float).reshape(-1,1)

    if self.transform:
      image = self.transform(image)
    return image,ydata

In [0]:
df = pd.read_csv('/content/drive/My Drive/Resonance/ChiefData.csv')

#transforms.RandomResizedCrop(224),
train,test = train_test_split(df,test_size=.2,random_state=2) 
train = ImageDataset(train,'/content/drive/My Drive/Resonance/Images',transform=transforms.Compose([transforms.Resize((64,64)),transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5,.5,.5],
                             std=[.5,.5,.5])]))


test = ImageDataset(test,'/content/drive/My Drive/Resonance/Images',transform=transforms.Compose([transforms.Resize((64,64)),transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),transforms.Normalize(mean=[.5,.5,.5],
                             std=[.5,.5,.5])
        ]))

train_loader = DataLoader(train, batch_size=4,
                        shuffle=False, num_workers=4,drop_last=True)
test_loader = DataLoader(test, batch_size=1,
                        shuffle=False, num_workers=4)

In [0]:
for i_batch, (image,y) in enumerate(train_loader):
    print(i_batch, image.size(),
          y.size())
    test = y
    if i_batch ==5:
      test = image
      break

    # observe 4th batch and stop.

0 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])
1 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])
2 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])
3 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])
4 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])
5 torch.Size([4, 3, 64, 64]) torch.Size([4, 1, 1])


In [0]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
import torch.nn as nn


class ChiefNet(nn.Module): 
    def __init__(self):
        super(ChiefNet, self).__init__()
        #feed forward layers
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=5,stride=1,padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size=5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.dropout = nn.Dropout()
        self.linear1 = nn.Linear(32*8*8,1000)
        self.linear2 = nn.Linear(1000,4)
        
        
        #activations
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() #Use sigmoid to convert the output into range (0,1)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer2(out)
        #out = self.layer2(out)
        #out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.linear1(out)
        out = self.linear2(out)
        return out
#torch.manual_seed(0)
net=ChiefNet()

#opt=torch.optim.SGD(net.parameters(),lr=1e-2,nesterov=True,momentum=.2)
opt = torch.optim.Adam(net.parameters(),lr=1e-3)

'''@torch.no_grad()
def init_weights(m):
    if type(m)==nn.Linear:
        m.weight.fill_(0)
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        #nn.init.xavier_uniform_(m.bias.data)
net.apply(init_weights)'''

loss = nn.CrossEntropyLoss()

train_loader = DataLoader(train, batch_size=96,
                        shuffle=True, num_workers=4,drop_last=True)
test_loader = DataLoader(test, batch_size=96,
                        shuffle=True, num_workers=4,drop_last=True)

loss_list = []
acc_list = []
total_step = len(train_loader)
num_epochs = 15
i = 0
test_loader = DataLoader(test, batch_size=len(test),
                        num_workers=4)
for epoch in range(num_epochs):
  print('epoch',epoch)
  if int(epoch+1) in [5,10,15,20,25,30]:    
    for i,(image,labels) in enumerate(test_loader):
      outputs = net(image)
      labels = labels.view(-1).long()
      _,predicted = torch.max(outputs.data,1)
      correct = (predicted==labels).sum().item()
      total = labels.size(0)
      acc = correct/total
      print('TEST ACC: ',acc)
  i = 0
  for i,(image,labels) in enumerate(train_loader):
    i+=1
    print('batch',i)
    outputs = net(image)
    labels = labels.view(-1).long()
    l = loss(outputs,labels)
    loss_list.append(l.item())

    opt.zero_grad()
    l.backward()
    opt.step()

    total = labels.size(0)
    _,predicted = torch.max(outputs.data,1)
    correct = (predicted==labels).sum().item()
    acc_list.append(correct/total)

    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, num_epochs, i + 1, total_step, l.item(),
                  correct / total) * 100)

In [0]:
torch.save(net, '/content/drive/My Drive/Resonance/ChiefNet.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ChiefNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
test_loader = DataLoader(test, batch_size=len(test),
                        num_workers=4)
for i,(image,labels) in enumerate(test_loader):
  outputs = net(image)
  labels = labels.view(-1).long()
  _,predicted = torch.max(outputs.data,1)
  correct = (predicted==labels).sum().item()
  total = labels.size(0)
  acc = correct/total

In [0]:
def inspect(num):
  outputs.data.numpy()[num]
  npimg = image.numpy()[num]
  plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
  t = pd.DataFrame(outputs.data.numpy()[num]).sort_values(0)
  #t['Color'] = cmatch['Unnamed: 0']
  print(labels[num].item())
  print('label',list(codes.loc[codes['0']==labels[num].item()]['Unnamed: 0'])[0])
  print('pred',list(codes.loc[codes['0']==predicted[num].item()]['Unnamed: 0'])[0])
  return t